In [ ]:
!pip install torch transformers coremltools

In [ ]:
from transformers import BertModel, BertTokenizer, BertConfig
import torch

enc = BertTokenizer.from_pretrained("google-bert/bert-base-uncased")

# Create dummy input
text = "[CLS] Who was Jim Henson ? [SEP] Jim Henson was a puppeteer [SEP]"
tokenized_text = enc.tokenize(text)

masked_index = 8
tokenized_text[masked_index] = "[MASK]"
indexed_tokens = enc.convert_tokens_to_ids(tokenized_text)
segments_ids = [0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1]

tokens_tensor = torch.tensor([indexed_tokens])
segments_tensors = torch.tensor([segments_ids])
dummy_input = [tokens_tensor, segments_tensors]

In [ ]:
# Trace the model
model = BertModel.from_pretrained("google-bert/bert-base-uncased", torchscript=True)
model.eval()
traced_model = torch.jit.trace(model, [tokens_tensor, segments_tensors])

In [ ]:
import coremltools as ct

# Convert into Core ML-compatible format
coreml_model = ct.convert(
  traced_model,
    inputs=[
      ct.TensorType(shape=dummy_input[0].shape, name="input_ids"),  # Renamed and matched to the first input
      ct.TensorType(shape=dummy_input[1].shape, name="attention_mask")  # Added the second input
  ],
  outputs=[
      ct.TensorType(name="last_hidden_state"),  # Output for the last hidden state
      ct.TensorType(name="pooler_output")  # Output for the pooler output
  ])

In [ ]:
# Save the Core ML model
coreml_model.save("bert-base-uncased.mlpackage")

In [ ]:
!zip -r bert-base-uncased.mlpackage.zip /content/bert-base-uncased.mlpackage

In [ ]:
from google.colab import files
files.download("bert-base-uncased.mlpackage.zip")